# Dự đoán cảm xúc từ các bài bình luận sản phẩm


Mục tiêu của notebook đầu tiên này là khám phá hồi quy Logistic và kỹ thuật đặc trưng với các hàm sklearn hiện có.

Trong notebook này, chúng ta sẽ sử dụng dữ liệu bình luận sản phẩm từ Amazon.com để dự đoán cảm xúc về một sản phẩm (từ bình luận) là tích cực hay tiêu cực.

* Sử dụng pandas để thực hiện một số kỹ thuật đặc trưng.
* Huấn luyện mô hình hồi quy Logistic để dự đoán cảm xúc của các bình luận về sản phẩm.
* Kiểm tra các trọng số (hệ số) của mô hình hồi quy Logistic đã huấn luyện.
* Đưa ra dự đoán (cả lớp và xác suất) của cảm xúc cho một bình luận sản phẩm mới.
* Viết hàm tính **accuracy** của mô hình biết trọng số hồi quy, các yếu tố dự đoán và nhãn ground truth.
* Kiểm tra hệ số của mô hình hồi quy Logistic và giải thích ý nghĩa của chúng.
* So sánh nhiều mô hình hồi quy Logistic.

Hãy bắt đầu thôi!

In [1]:
# Import một số thư viện

import pandas
import numpy as np
from sklearn.model_selection import train_test_split

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
# Import dữ liệu amazon_baby.csv vào pandas dataframe
products_df = pandas.read_csv('amazon_baby.csv')

### 1. Tiền xử lý dữ liệu
Trước khi sử dụng dữ liệu đánh giá, chúng ta cần thực hiện tiền xử lý
<br>
Trước tiên, loại bỏ dấu câu khỏi các từ
<br>
Ví dụ: **good!** --> **good**, **resturant.** --> **resturant**

In [4]:
def remove_punctuation(text):
    import string
    return text.translate(string.punctuation)

products_df = products_df.fillna({'review':''})  # điền vào các N/A ở cột review
products_df['review_clean'] = products_df['review'].apply(remove_punctuation)

Chúng ta sẽ bỏ qua tất cả các bình luận có rating = 3 vì chúng có cảm xúc trung lập.

In [5]:
products_df = products_df[products_df['rating'] != 3]

Bây giờ, chúng ta sẽ chỉ định các bình luận có rating từ 4 trở lên là bình luận tích cực, trong khi các bình luận từ 2 trở xuống là tiêu cực. Đối với cột sentiment, chúng ta sử dụng +1 cho nhãn positive class (lớp tích cực) và -1 cho nhãn negative class (lớp tiêu cực). Chúng ta nên tạo một hàm ẩn danh chuyển đổi rating thành class label, sau đó áp dụng hàm đó cho mọi phần tử trong cột rating. Trong Pandas, chúng ta sẽ sử dụng appl ():

In [6]:
products_df['sentiment'] = products_df['rating'].apply(lambda rating : +1 if rating > 3 else -1)

Hãy thực hiện phân tách train/test với 80% dữ liệu trong tập huấn luyện và 20% dữ liệu trong tập kiểm tra.
Chúng ta sẽ thiết lập seed để đảm bảo nhận được các kết quả tương tự.

In [7]:
np.random.seed(1)

train_data, test_data = train_test_split(products_df, test_size=0.2)

### 2. Biểu diễn đặc trưng
 Bây giờ, chúng ta sẽ tính số từ cho mỗi từ xuất hiện trong các bình luận. Vectơ đếm từ thường được gọi là đặc trưng túi từ. Vì hầu hết các từ chỉ xuất hiện trong một số bình luận nên các vectơ đếm từ khá thưa thớt. Do đó, scikit-learn và nhiều công cụ khác sử dụng ma trận thưa thớt để lưu trữ tập hợp các vectơ đếm từ.
<br>
Có thể tham khảo [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) từ tài liệu scikit.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Sử dụng mẫu token này để giữ các từ có một chữ cái
# Trước tiên, tìm hiểu từ vựng từ dữ liệu huấn luyện và gán cột cho từ
# Sau đó chuyển đổi dữ liệu huấn luyện thành ma trận thưa thớt
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Tiếp theo, chuyển đổi dữ liệu kiểm tra thành ma trận thưa thớt, sử dụng cùng một ánh xạ từ-cột
test_matrix = vectorizer.transform(test_data['review_clean'])


### 3. Huấn luyện mô hình hồi quy logistic

Tìm hiểu phân loại hồi quy logistic bằng cách sử dụng dữ liệu huấn luyện. Chúng ta đang sử dụng scikit-learn, nên hãy tạo một instance của [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), sau đó gọi phương thức fit() để huấn luyện phân loại. Mô hình này nên sử dụng ma trận đếm từ thưa thớt (**train_matrix**) làm đặc trưng và cột **sentiment** của **train_data** làm mục tiêu. Sử dụng các giá trị mặc định cho các tham số khác.

In [13]:
### VIẾT CODE Ở ĐÂY ###
from sklearn.linear_model import LogisticRegression

X, y = train_matrix, train_data['sentiment']

clf = LogisticRegression().fit(X,y)
### VIẾT CODE Ở ĐÂY ###

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Kiểm tra train accuracy.
<br>
Nên thu được train accuracy là: 0.9474291796913067

In [14]:
print ('Train accuracy: {}'.format(clf.score(X, y)))

Train accuracy: 0.9475116378437943


Kiểm tra test accuracy. 
Nên thu được test accuracy là: 0.9345746754220263

In [15]:
print ('Test accuracy: {}'.format(clf.score(test_matrix, test_data['sentiment'])))

Test accuracy: 0.9344547389883362


In [16]:
sum(1 for c in clf.coef_[0] if c >= 0)

40188

**Quiz**: Có bao nhiêu trọng số >= 0?
<br>
**Đáp án**:
<br>
`sum(1 for c in clf.coef_[0] if c >= 0)`

### 4. Đưa ra dự đoán

Trước tiên, hãy tính score cho 6 mẫu từ **test_data**.
<br>
Nhớ lại từ bài giảng: $$\mbox{score}_i = \mathbf{w}^\intercal h(\mathbf{x}_i)$$

In [17]:
sample_test_data = test_data[105:110]
print(sample_test_data['review_clean'])
print(sample_test_data['rating'])
print(sample_test_data['sentiment'])

76151     It came to me with marks on it.I recently went...
164181    How did we survive before discovering Kidzikoo...
108065    My baby loves this jumper! He'll stay playing ...
176539    Looked at this set at Walmart for $50 and coul...
56851     Product was not working when received.  Batter...
Name: review_clean, dtype: object
76151     2
164181    5
108065    5
176539    5
56851     1
Name: rating, dtype: int64
76151    -1
164181    1
108065    1
176539    1
56851    -1
Name: sentiment, dtype: int64


In [18]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = clf.decision_function(sample_test_matrix)
print (scores)

[-6.31521729  6.11838966  4.3730779   2.0046506  -5.6699474 ]


Các score này được sử dụng để dự đoán lớp như sau:
<br>
$$\hat{y}_i = \begin{cases} +1 & \text{if } \mathbf{w}^\intercal h(\mathbf{x}_i) > 0\\ -1 &\text{if } \mathbf{w}^\intercal h(\mathbf{x}_i) \leq 0 \end{cases}$$

Chúng ta có thể sử dụng hàm predict() từ scikit-learn.

In [19]:
print (clf.predict(sample_test_matrix))

[-1  1  1  1 -1]


 Nhớ lại từ bài giảng, chúng ta cũng có thể tính xác suất dự đoán từ score sử dụng:
 <br>
 $$P(y_i = +1 | \mathbf{x}_i, \mathbf{w}) = \dfrac{1}{1+\exp{(-\mathbf{w}^\intercal h(\mathbf{x}_i))}}$$

Chúng ta có thể sử dụng hàm predict_proba() từ scikit-learn.

In [21]:
1/(1+np.exp(-6.31521729))

0.9981946922971531

In [20]:
print (clf.predict_proba(sample_test_matrix))
# LƯU Ý: P(yi=+1|xi,w) thể hiện trong cột thứ hai, cột thứ nhất thể hiện P(yi=-1|xi,w)

[[0.99819469 0.00180531]
 [0.00219716 0.99780284]
 [0.01245527 0.98754473]
 [0.1187155  0.8812845 ]
 [0.9965638  0.0034362 ]]


Tính accuracy:
<br>
$$\mbox{accuracy} = \dfrac{\mbox{# correctly classified examples}}{\mbox{# total examples}}$$
<br>
Chúng ta có thể sử dụng hàm score() từ scikit-learn.

In [22]:
# Accuracy của 6 mẫu trên :DD
print (clf.score(sample_test_matrix, sample_test_data['sentiment']))

1.0


In [23]:
# Accuracy cho toàn bộ test_data
print ('Test accuracy: {}'.format(clf.score(test_matrix, test_data['sentiment'])))

Test accuracy: 0.9344547389883362
